In [270]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import itertools
import sys
import copy

sys.path.append("..")

from utility_functions import (calculate_model_performance,
                               plot_ROC,
                               one_hot_encode,
                               split_data_as,
                               grid_search,
                               shuffled,
                               timeit)

EPSILON = 10e-08


def get_shapes(any_):
    for array in any_:
        try:
            print(array.shape)
        except:
            print("NONE")
    print("\n")


# ============= ACTIVATION FUNCTIONS ===============#

def sigmoid(Z, prime=False):
    # np.
    if prime:
        return sigmoid(Z) * (1 - sigmoid(Z))
    return 1 / (1 + np.exp(-Z))


def linear(Z, prime=False):
    if prime:
        return np.ones_like(Z)
    return Z


def relu(Z, alpha=0.01, prime=False):
    if prime:
        Z_relu = np.ones_like(Z, dtype=np.float64)
        Z_relu[Z < 0] = alpha
        return Z_relu
    return np.where(Z < 0, alpha * Z, Z)


def tanh(Z, prime=False):
    # np.tanh() could be used directly to speed this up
    if prime:
        return 1 - np.power(tanh(Z), 2)
    return (2 / (1 + np.exp(-2 * Z))) - 1


def elu(Z, prime=False):
    # https://mlfromscratch.com/activation-functions-explained/#/
    alpha = 0.2
    if prime:
        return np.where(Z < 0, alpha * (np.exp(Z)), 1)
    return np.where(Z < 0, alpha * (np.exp(Z) - 1), Z)


def softmax(Z, prime=False):
    # https://deepnotes.io/softmax-crossentropy
    # max(Z) term is added to stabilise the function.
    exps = np.exp(Z - np.max(Z))
    return exps / np.sum(exps, axis=0)


# References
# https://mc.ai/multilayered-neural-network-from-scratch-using-python/
# https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
# https://www.coursera.org/learn/machine-learning/home/week/5
# https://www.coursera.org/specializations/deep-learning
# https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network.py
# https://github.com/JWarmenhoven/Coursera-Machine-Learning

class NeuralNetwork:

    def __init__(
            self,
            input_layer: tuple,
            hidden_layer: list,  # list of tuples
            output_layer: int,
            batch_size=16,
            alpha=0.01,
            optimizer="SGD",
            weight_initialisation = "he_uniform",
            penalty=None,
            lambd=0.01,
            keep_prob = None,
            epoch=500,
            random_state=42,
            verbose=True,
            metrics="accuracy"
    ):
        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.output_layer = output_layer
        self.mini_batch_size = batch_size
        self.alpha = alpha
        self.optimizer = optimizer
        self.weight_initialisation = weight_initialisation
        self.penalty = penalty
        self.lambd = lambd
        self.keep_prob = keep_prob
        # dropout: http://jmlr.org/papers/volume15/srivastava14a.old/srivastava14a.pdf
        self.dropout = True if isinstance(self.keep_prob, float) else False
        self.epoch = epoch
        self.seed = random_state
        self.verbose = verbose
        self.metrics = metrics
        self.layers = len(self.weight_set_dimensions) + 1
        self.EPSILON = 10e-10
        self.best_score = 0
        self.best_W = np.empty_like(range(self.layers), dtype=object) if verbose else None
        self.best_B = np.empty_like(range(self.layers), dtype=object) if verbose else None
        
        
    def __str__(self):
        parameters = (
            "Input layer: {0}\n"
            "Hidden layer: {1}\n"
            "Output layer: {2}\n"
            "Batch size: {3}\n"
            "Learning rate: {4}\n"
            "Epoch: {5}\n"
            "Seed: {6}\n"
            "Verbose: {7}\n"
            "Metric: {8}"
        ).format(
            self.input_layer,
            " - ".join(map(str, self.hidden_layer)),
            self.output_layer,
            self.mini_batch_size,
            self.alpha,
            self.epoch,
            self.seed,
            self.verbose,
            self.metrics
        )
        return parameters

    def get_A(self, X, predict=True):
        A, _ = self.forwardpass(X, predict=predict)
        return A

    def get_Z(self, X, predict=True):
        _, Z = self.forwardpass(X, predict=predict)
        return Z
    
    # ============== LOSS FUNCTIONS ===============#

    # https://deepnotes.io/softmax-crossentropy

    def calculate_error(self, Y, Y_hat):
        # Y and Y_hat should be in the form of (no_of_classes, no_of_training_examples)
        cost = -np.sum(Y * np.log(Y_hat + EPSILON)) / self.m
        if self.penalty == "l1":
            for layer in range(1, self.layers):
                cost += np.sum(np.abs(self.W[layer])) * self.lambd / (2 * self.m)
        elif self.penalty == "l2":
            for layer in range(1, self.layers):
                cost += np.sum(np.square(self.W[layer])) * self.lambd / (2 * self.m)
        return cost


    def display_information(self, X, Y, X_test, Y_test, epoch_no):
        model_performance_metrics_train = calculate_model_performance(
            np.argmax(Y, axis=0),
            self.predict(X)
        )
        
        model_performance_metrics_test = calculate_model_performance(
            np.argmax(Y_test, axis=0),
            self.predict(X_test)
        )
        print("train_{0}: {1} ____ test_{0}: {2}  - epoch {3}    iteration {4} - loss {5}".format(
            self.metrics,
            np.round(model_performance_metrics_train[self.metrics], 5),
            np.round(model_performance_metrics_test[self.metrics], 5),
            epoch_no,
            self.no_of_iterations,
            np.round(self.calculate_error(Y,
                            self.get_A(X)[-1]), 10)
        ))

        self.get_best_scoring_weight(model_performance_metrics_test)
        
    def get_best_scoring_weight(self, test_performance):
        score = test_performance[self.metrics]
        
        # self.best_score is initialised with 0 upon the construction of the class
        if score > self.best_score:
            # start from the first element since first element is None, deepcopy
            # throws an error
            self.best_W[1:] = copy.deepcopy(self.W[1:])
            self.best_B[1:] = copy.deepcopy(self.B[1:])
            self.best_score = score
            # print(self.best_score)

    def get_dimensions_and_activations(self):
        self.dimensions = []
        self.activation_functions = []

        self.dimensions.append(self.input_layer[0])
        self.activation_functions.append(self.input_layer[1])

        for dim, act_func in self.hidden_layer:
            self.dimensions.append(dim)
            self.activation_functions.append(act_func)

        self.dimensions.append(self.output_layer)

    @property
    def weight_set_dimensions(self):
        self.get_dimensions_and_activations()
        a, b = itertools.tee(self.dimensions[::-1])
        next(b, None)
        weight_set_dimensions = list(zip(a, b))[::-1]
        return weight_set_dimensions

    def initialise_weights(self, layer=None):
        self.W = np.empty_like(range(self.layers), dtype=object)
        self.B = np.empty_like(range(self.layers), dtype=object)
        np.random.seed(self.seed)

        for layer, (y, x) in zip(range(1, self.layers), self.weight_set_dimensions):
            # x = (layer - 1) *** fan_in
            # y = (layer)  *** fan_out
            
            # difference between randn and normal:
            # https://stackoverflow.com/questions/21738383/python-difference-between-randn-and-normal

            if isinstance(self.weight_initialisation, dict):
                if self.weight_initialisation["method"] == "random_normal":
                    mean = self.weight_initialisation["mean"]
                    standard_deviation = self.weight_initialisation["standard_deviation"]

                    self.W[layer] = np.random.normal(mean, standard_deviation, size=(y, x))
                    self.B[layer] = np.random.normal(mean, standard_deviation, size=(y, 1))

                elif self.weight_initialisation["method"] == "random_uniform":
                    min_ = self.weight_initialisation["min"]
                    max_ = self.weight_initialisation["max"]

                    self.W[layer] = np.random.uniform(min_, max_, size=(y, x))
                    self.B[layer] = np.random.uniform(min_, max_, size=(y, 1))

            elif self.weight_initialisation == "zeros":
                self.W[layer] = np.zeros((y, x))
                self.B[layer] = np.zeros((y, 1))

            elif self.weight_initialisation == "ones":
                self.W[layer] = np.ones((y, x))
                self.B[layer] = np.ones((y, 1))

            elif self.weight_initialisation == "he_normal":
                # https://arxiv.org/abs/1502.01852
                # https://keras.rstudio.com/reference/initializer_he_normal.html
                self.W[layer] = np.random.normal(
                    loc=0,
                    scale=np.sqrt(2 / x),
                    size=(y, x)
                )
                self.B[layer] = np.zeros((y, 1))
                
            elif self.weight_initialisation == "he_uniform":
                # https://docs.chainer.org/en/stable/reference/generated/chainer.initializers.HeUniform.html
                # https://keras.rstudio.com/reference/initializer_he_uniform.html
                limit = np.sqrt(6 / x)

                self.W[layer] = np.random.uniform(
                    low=-limit,
                    high=limit,
                    size=(y, x)
                )
                self.B[layer] = np.zeros((y, 1))
            
            elif self.weight_initialisation == "xavier_normal":
                # Glorot normal initializer, also called Xavier normal initializer.
                # https://keras.rstudio.com/reference/initializer_glorot_normal.html
                self.W[layer] = np.random.normal(
                    loc=0,
                    scale=np.sqrt(2 / (x + y)),
                    size=(y, x)
                )
                self.B[layer] = np.zeros((y, 1))
                
            elif self.weight_initialisation == "xavier_uniform":
                # https://keras.rstudio.com/reference/initializer_glorot_uniform.html
                limit = np.sqrt(6 / (x + y))

                self.W[layer] = np.random.uniform(
                    low=-limit,
                    high=limit,
                    size=(y, x)
                )                
                self.B[layer] = np.zeros((y, 1))

    def forwardpass(self, X, predict=False):
        Z = np.empty_like(range(self.layers), dtype=object)
        A = np.empty_like(range(self.layers), dtype=object)
        A[0] = X

        for layer in range(1, self.layers):
            # activation_function starts from 0 whereas layer starts from 1
            active_function = self.activation_functions[layer - 1]
            arg_to_pass_to_eval = "(Z[layer])"

            Z[layer] = self.W[layer] @ A[layer - 1] + self.B[layer]
            A[layer] = eval(active_function + arg_to_pass_to_eval)
            
            # dropout is only applied to first hidden layer
            # https://www.kaggle.com/mtax687/dropout-regularization-of-neural-net-using-numpy
            if self.dropout and layer == 1 and not predict:
                self.D = np.random.randn(A[layer].shape[0], A[layer].shape[1])
                self.D = (self.D < self.keep_prob)
                A[layer] = np.multiply(A[layer], self.D) / self.keep_prob

        return A, Z

    def backpropagation(self, Y, A, Z):
        self.delta = np.empty_like(range(self.layers), dtype=object)

        self.gradient_W = np.empty_like(range(self.layers), dtype=object)
        self.gradient_B = np.empty_like(range(self.layers), dtype=object)

        self.delta[-1] = A[-1] - Y

        # We substract 1 here as delta_final is calculated seperately above
        for layer in reversed(range(1, self.layers - 1)):
            # 1 is substracted from layer as activation_functions start indexing from 0
            active_function = self.activation_functions[layer - 1]
            arg_to_pass_to_eval = "(Z[layer], prime=True)"

            DA = self.W[layer + 1].T @ self.delta[layer + 1]

            # If dropout is applied
            if self.dropout and layer == 1:
                DA = np.multiply(DA, self.D) / self.keep_prob


            self.delta[layer] = (
                    DA *
                    eval(active_function + arg_to_pass_to_eval)
            )

        for layer in range(1, self.layers):
            self.gradient_W[layer] = (self.delta[layer] @ A[layer - 1].T) / self.m
            self.gradient_B[layer] = np.sum(self.delta[layer], axis=1, keepdims=True) / self.m
            
            if self.penalty == "l1":
                # https://towardsdatascience.com/only-numpy-implementing-different-combination-of-l1-norm-l2-norm-l1-regularization-and-14b01a9773b
                self.gradient_W[layer] += np.where(self.W[layer] < 0, -1, 1) * (self.lambd / self.m)
            elif self.penalty == "l2":
                self.gradient_W[layer] += self.W[layer] * (self.lambd / self.m)

        self.update_weights()

    def update_weights(self):
        if self.optimizer == "SGD":
            for layer in range(1, self.layers):
                self.W[layer] -= self.alpha * self.gradient_W[layer]
                self.B[layer] -= self.alpha * self.gradient_B[layer]

        elif self.optimizer["method"] == "SGDM":
            for layer in range(1, self.layers):
                beta = self.optimizer["beta"]
                self.v_dw[layer] = beta * self.v_dw[layer] + (1 - beta) * self.gradient_W[layer]
                self.v_db[layer] = beta * self.v_db[layer] + (1 - beta) * self.gradient_B[layer]

                self.W[layer] -= self.alpha * self.v_dw[layer]
                self.B[layer] -= self.alpha * self.v_db[layer]

        elif self.optimizer["method"] == "RMSP":
            for layer in range(1, self.layers):
                beta = self.optimizer["beta"]
                self.s_dw[layer] = beta * self.s_dw[layer] + (1 - beta) * np.square(self.gradient_W[layer])
                self.s_db[layer] = beta * self.s_db[layer] + (1 - beta) * np.square(self.gradient_B[layer])

                w_rms_grad = self.gradient_W[layer] / (np.sqrt(self.s_dw[layer]) + self.EPSILON)
                b_rms_grad = self.gradient_B[layer] / (np.sqrt(self.s_db[layer]) + self.EPSILON)

                self.W[layer] -= self.alpha * w_rms_grad
                self.B[layer] -= self.alpha * b_rms_grad

        elif self.optimizer["method"] == "ADAM":
            # EWA: Exponential weighted average
            # ToDo: Check if bias correction is necessary. The EWA will be inaccurate initially,
            # but it shouldn't take many iterations to compute correct EWA.
            for layer in range(1, self.layers):
                beta1 = self.optimizer["beta1"]
                beta2 = self.optimizer["beta2"]
                self.v_dw[layer] = beta1 * self.v_dw[layer] + (1 - beta1) * self.gradient_W[layer]
                self.v_db[layer] = beta1 * self.v_db[layer] + (1 - beta1) * self.gradient_B[layer]

                self.s_dw[layer] = beta2 * self.s_dw[layer] + (1 - beta2) * np.square(self.gradient_W[layer])
                self.s_db[layer] = beta2 * self.s_db[layer] + (1 - beta2) * np.square(self.gradient_B[layer])

                v_dw_corrected = self.v_dw[layer] / (1 - beta1 ** self.no_of_iterations)
                s_dw_corrected = self.s_dw[layer] / (1 - beta2 ** self.no_of_iterations)

                v_db_corrected = self.v_db[layer] / (1 - beta1 ** self.no_of_iterations)
                s_db_corrected = self.s_db[layer] / (1 - beta2 ** self.no_of_iterations)

                self.W[layer] -= self.alpha * (v_dw_corrected / (np.sqrt(s_dw_corrected) + self.EPSILON))
                self.B[layer] -= self.alpha * (v_db_corrected / (np.sqrt(s_db_corrected) + self.EPSILON))


    def initialise_cache(self):
        self.v_dw = np.empty_like(range(self.layers), dtype=object)
        self.v_db = np.empty_like(range(self.layers), dtype=object)
    
        self.s_dw = np.empty_like(range(self.layers), dtype=object)
        self.s_db = np.empty_like(range(self.layers), dtype=object)

        for layer, (y, x) in zip(range(1, self.layers), self.weight_set_dimensions):
            self.v_dw[layer] = np.zeros((y, x))
            self.v_db[layer] = np.zeros((y, 1))
            
            self.s_dw[layer] = np.zeros((y, x))
            self.s_db[layer] = np.zeros((y, 1))

    @timeit
    def fit(self, X, Y, X_test, Y_test):
        self.m = X.shape[1] # where (no_of_features, no_of_training_examples)
        self.initialise_weights()
        self.initialise_cache()

        # By default the method is SGD(Stochastic Gradient Descent) if one wishes to use
        # the whole batch, simply pass the number of traning examples available as the
        # batch size when instantiating the class
        self.no_of_iterations = 0
        shuffled = np.arange(self.m)
        if self.verbose:
            print("Initialising weights...")
            print("Starting the training...")
            print("Initial cost: %.10f\n" % self.calculate_error(Y, self.get_A(X)[-1]))
        for epoch_no in range(1, self.epoch + 1):
            np.random.shuffle(shuffled)
            X_shuffled = X[:, shuffled]
            Y_shuffled = Y[:, shuffled]
            for i in range(0, self.m, self.mini_batch_size):
                self.no_of_iterations += 1
                X_mini_batch = X_shuffled[:, i: i + self.mini_batch_size]
                Y_mini_batch = Y_shuffled[:, i: i + self.mini_batch_size]

                A, Z = self.forwardpass(X_mini_batch)
                self.backpropagation(Y_mini_batch, A, Z)
                if self.no_of_iterations % 100 == 0 and self.verbose:
                    self.display_information(X, Y, X_test, Y_test, epoch_no)

    def predict(
            self,
            X: np.ndarray,
            apply_dropout=False,
            return_prob_matrix=False,
    ):
        """Predict the output given the training data.

            Returns the predicted values in two forms:

            1.either by picking up the highest value along the columns for every row,
                i.e. "np.argmax(self.A[-1].T, axis=1)"
            2.or by returning a matrix that is in the shape of Y.T where each column
                represents the probability of the instance belonging to that class.
                Please note that every column in Y.T represents a class. To be able to
                return the probability matrix, the final activation function must be
                softmax!
                i.e. "array([0.9650488423, 0.0354737543, 0.0005225966])"

        Args:
            X (numpy.ndarray): Training set in the shape of
                (no_of_features, no_of_training examples).
            return_prob_matrix (bool, optional): Returns the probability matrix if True.
                Defaults to False.

        Returns:
            numpy.ndarray:

            if return_prob_matrix is False, the output is in the shape of
                (no_of_training_examples, 1)
            if return_prob_matrix is True, the output is in the shape of
                (no_of_training_examples, no_of_features)
        """
        # here predict means if forwardpass is called from predict method
        # if so, there is an option to apply or not apply dropout
        predict = np.invert(apply_dropout)
        A, Z = self.forwardpass(X, predict=predict)
        if return_prob_matrix:
            np.set_printoptions(precision=10, suppress=True)
            return A[-1].T
        return np.argmax(A[-1].T, axis=1)

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()

x = data.data[:,[0,2]]
y = data.target

X = x.T
Y = one_hot_encode(y).T

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import initializers
from keras.layers import Dropout
import time

In [ ]:
start = time.time()
model_keras = Sequential()
model_keras.add(Dense(4, input_dim=2, activation='relu'))
model_keras.add(Dropout(0.5))
model_keras.add(Dense(4, activation='relu'))
model_keras.add(Dense(3, activation='softmax'))
adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
model_keras.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_keras.fit(X.T,Y.T, batch_size=10, epochs=100)
end = time.time()
print(end - start)

# Testing with benchmark datasets

## 1.Iris Dataset

In [271]:
from sklearn.datasets import load_iris
data = load_iris()

x = data.data[:,[0,2]]
y = data.target

dataset_train, dataset_test = split_data_as(x, y, train=0.8, test=0.2)

X_train = dataset_train[:, :-1].T
Y_train = one_hot_encode(dataset_train[:, -1]).T

X_test = dataset_test[:, :-1].T
Y_test = one_hot_encode(dataset_test[:, -1]).T


# X = x.T
# Y = one_hot_encode(y).T

Splitting the dataset as test and train...


In [272]:
print(X_train.shape)
print(Y_train.shape)

(2, 30)
(3, 30)


In [273]:
model = NeuralNetwork(
    input_layer=(2, 'relu'),
    hidden_layer=[(4,'relu'),(4,'softmax')],
    output_layer=3,
    batch_size=10,
    optimizer={"method": "ADAM", "beta1": 0.9, "beta2": 0.999},
    weight_initialisation = "xavier_normal",
    keep_prob=1,
    penalty=None,
    epoch=100,
    alpha=0.01
)

model.fit(X_train, Y_train, X_test, Y_test)

Initialising weights...
Starting the training...
Initial cost: 1.0962094968

train_accuracy: 73.33333 ____ test_accuracy: 75.0  - epoch 34    iteration 100 - loss 0.4243553343
train_accuracy: 96.66667 ____ test_accuracy: 93.33333  - epoch 67    iteration 200 - loss 0.1436908921
train_accuracy: 96.66667 ____ test_accuracy: 92.5  - epoch 100    iteration 300 - loss 0.1047364436
func:'fit' -- took: 0.1407 sec


In [274]:
calculate_model_performance(np.argmax(Y_test, axis=0),
                           model.predict(X_test))

{'F1': 89.15662650337757,
 'accuracy': 92.49999999922916,
 'false_positive_rate': 8.641975308535283,
 'precision': 84.09090908899795,
 'prevalence': 32.49999999972917,
 'sensitivity/recall': 94.87179486936228,
 'specificity': 91.35802469023014}

In [ ]:
# results_dict_all_models, results_average_dict, models = grid_search(
#     x,
#     y,
#     clf=NeuralNetwork,
#     lst_metrics=["F1", "accuracy"],
#     sort_by = "accuracy",
#     n_folds=5,
#     dict_param_grid={
#         'batch_size': [8, 16, 32],
#         'input_layer': [(2, 'relu')],
#         'hidden_layer': [
#             [(4,'relu'), (4,'softmax')],
#             [(4,'sigmoid'),(4,'softmax')]
#         ],
#         'optimizer': [
#             {
#                 "method": "RMSP",
#                 "beta": 0.9
#             }
#         ],
#         'output_layer': [3],
#         'alpha': [0.001],
#         'verbose': [False],
#         'epoch': [1000]
#     }
# )

In [276]:
# %matplotlib inline
import matplotlib.pyplot as plt
dt = data.data[:,[0,2]]
x_min, x_max = dt[:, 0].min() - 1, dt[:, 0].max() + 1
y_min, y_max = dt[:, 1].min() - 1, dt[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx.ravel(), yy.ravel()].T, apply_dropout=False)

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx, yy, Z,alpha=0.4)
#plt.axis('off')

plt.scatter(dt[:, 0], dt[:, 1], c=y,s=20, edgecolor='k')
plt.xlabel('sepal length')
plt.ylabel('petal length')
plt.show()

C:\Anaconda3\lib\site-packages\numpy\ma\core.py:6461: MaskedArrayFutureWarning: In the future the default for ma.maximum.reduce will be axis=0, not the current None, to match np.maximum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
C:\Anaconda3\lib\site-packages\numpy\ma\core.py:6461: MaskedArrayFutureWarning: In the future the default for ma.minimum.reduce will be axis=0, not the current None, to match np.minimum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)


## 2.Make Moons dataset

In [ ]:
from sklearn.datasets import make_moons

x,y =make_moons(n_samples=1500, noise=.05)
X = x.T
Y = one_hot_encode(y).T

print(X.shape)
print(Y.shape)

In [ ]:
model = NeuralNetwork(
    input_layer=(X.shape[0], 'relu'),
    hidden_layer=[(10,'relu'), (4,'softmax')],
    output_layer=Y.shape[0],
    batch_size=8,
    optimizer=
    {
        "method": "ADAM",
        "beta1": 0.9,
        "beta2": 0.999
    },
    penalty = "l2",
    keep_prob=0.5,
    lambd=0.001,
    epoch=250,
    alpha=0.01
)

model.fit(X,Y)

In [ ]:
### Decision Boundaries
%matplotlib inline
import matplotlib.pyplot as plt
dt = x
x_min, x_max = dt[:, 0].min() - 0.5, dt[:, 0].max() + 0.5
y_min, y_max = dt[:, 1].min() - 0.5, dt[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx.ravel(), yy.ravel()].T, apply_dropout=False) 

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx, yy, Z,alpha=0.4)
#plt.axis('off')
plt.scatter(dt[:, 0], dt[:, 1], c=y, s=20, edgecolor='k')
plt.title('Decision Boundaries')

## 3.Andrew NG Assignment 2 Dataset

In [205]:
ex2data2 = np.loadtxt("../ex2/data/ex2data2.txt", delimiter=",")

x = ex2data2[:, :-1]
y = ex2data2[:, -1]

X = x.T
Y = one_hot_encode(y).T

In [206]:
print(X.shape)
print(Y.shape)

(2, 118)
(2, 118)


In [ ]:
model = NeuralNetwork(
    input_layer=(X.shape[0], 'relu'),
    hidden_layer=[(8,'relu'), (4,'softmax')],
    output_layer=Y.shape[0],
    batch_size=8,
    optimizer={"method": "ADAM", "beta1": 0.9, "beta2": 0.999},
    keep_prob=0.8,
    epoch=1500,
    alpha=0.01
)

model.fit(X,Y)

In [ ]:
# results_dict_all_models, results_average_dict, models = grid_search_stratified(
#     x,
#     y,
#     clf=NeuralNetwork,
#     metrics=["F1", "accuracy"],
#     sort_by = "accuracy",
#     n_fold=6,
#     param_grid_dict={
#         'batch_size': [16, 32],
#         'input_layer': [(2, 'relu')],
#         'hidden_layer': [
#             [(4,'relu'), (4,'relu'), (4,'softmax')],
#             [(4,'sigmoid'),(4,'softmax')]
#         ],
#         'output_layer': [2],
#         'alpha': [2, 4],
#         'verbose': [False],
#         'epoch': [5000]
#     }
# )

In [ ]:
results_average_dict

In [ ]:
print(models["model_5"])

In [ ]:
### Decision Boundaries
%matplotlib inline
import matplotlib.pyplot as plt
data = ex2data2

x1_min, x1_max = data[:, 0].min() - 0.3, data[:, 0].max() + 0.3,
x2_min, x2_max = data[:, 1].min() - 0.3, data[:, 1].max() + 0.3,
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max), np.linspace(x2_min, x2_max))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx1.ravel(), xx2.ravel()].T, apply_dropout=False) 

negatives = ex2data2[ex2data2[:, -1] == 0]
positives = ex2data2[ex2data2[:, -1] == 1]

# Put the result into a color plot
Z = Z.reshape(xx1.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx1, xx2, Z,alpha=0.4)
#plt.axis('off')
plt.scatter(negatives[:, 0], negatives[:, 1],s=50, color='k')
plt.scatter(positives[:, 0], positives[:, 1],s=50, color='r')
plt.title('Decision Boundaries')

plt.contour(xx1, xx2, Z, [0.5], linewidths=2, colors="g")

# MNIST dataset

In [277]:
from scipy.io import loadmat
data = loadmat('../ex3/data/ex3data1.mat')
data.keys()

dict_keys(['X', '__header__', '__version__', '__globals__', 'y'])

In [278]:
x = data["X"]
y = data["y"]
y[y==10] = 0

In [279]:
dataset_test, dataset_validation, dataset_train = split_data_as(x, y, train=0.8, test=0.1, validation=0.1)
X_train = dataset_train[:, :-1].T
Y_train = one_hot_encode(dataset_train[:, -1]).T

X_test = dataset_test[:, :-1].T
Y_test = one_hot_encode(dataset_test[:, -1]).T

X_validation = dataset_validation[:, :-1].T
Y_validation = one_hot_encode(dataset_validation[:, -1]).T

Splitting the dataset as test, validation and train...


In [280]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20,20))

sample = np.random.choice(data["X"].shape[0], 20)
ax.imshow(data["X"][sample,1:].reshape(-1,20).T)
ax.axis('off');

In [281]:
# http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354

model = NeuralNetwork(
    input_layer=(X_train.shape[0], 'relu'),
    hidden_layer=[(200,'relu'),(100,'relu'), (50,'relu'), (25, 'relu'), (4,'softmax')],
    output_layer=Y_train.shape[0],
    batch_size=50,
    optimizer={"method": "ADAM", "beta1": 0.9, "beta2": 0.999},
    weight_initialisation = "xavier_uniform",
    penalty=None,
    metrics="accuracy",
    epoch=100,
    alpha=0.01,
    verbose=True
)

model.fit(X_train, Y_train, X_test, Y_test)

Initialising weights...
Starting the training...
Initial cost: 2.3007645699

train_accuracy: 55.325 ____ test_accuracy: 53.2  - epoch 2    iteration 100 - loss 1.4388698328
train_accuracy: 78.125 ____ test_accuracy: 72.8  - epoch 3    iteration 200 - loss 0.7803600256
train_accuracy: 86.2 ____ test_accuracy: 80.6  - epoch 4    iteration 300 - loss 0.4899953164
train_accuracy: 88.425 ____ test_accuracy: 83.8  - epoch 5    iteration 400 - loss 0.410429103
train_accuracy: 93.4 ____ test_accuracy: 89.0  - epoch 7    iteration 500 - loss 0.2715323505
train_accuracy: 94.95 ____ test_accuracy: 88.4  - epoch 8    iteration 600 - loss 0.1948008794
train_accuracy: 95.275 ____ test_accuracy: 89.2  - epoch 9    iteration 700 - loss 0.2017770758
train_accuracy: 96.45 ____ test_accuracy: 90.6  - epoch 10    iteration 800 - loss 0.1300057845
train_accuracy: 96.875 ____ test_accuracy: 91.6  - epoch 12    iteration 900 - loss 0.111093967
train_accuracy: 97.55 ____ test_accuracy: 91.6  - epoch 13    ite

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:72: RuntimeWarning: invalid value encountered in true_divide


train_accuracy: 71.075 ____ test_accuracy: 90.0  - epoch 30    iteration 2400 - loss nan
train_accuracy: 99.125 ____ test_accuracy: 91.8  - epoch 32    iteration 2500 - loss 0.0328457264
train_accuracy: 99.625 ____ test_accuracy: 92.6  - epoch 33    iteration 2600 - loss 0.0137026421
train_accuracy: 99.675 ____ test_accuracy: 91.8  - epoch 34    iteration 2700 - loss 0.0125033163
train_accuracy: 99.75 ____ test_accuracy: 92.4  - epoch 35    iteration 2800 - loss 0.0091953845
train_accuracy: 99.725 ____ test_accuracy: 92.2  - epoch 37    iteration 2900 - loss 0.0064074401
train_accuracy: 99.775 ____ test_accuracy: 92.0  - epoch 38    iteration 3000 - loss 0.0048296968
train_accuracy: 99.825 ____ test_accuracy: 91.8  - epoch 39    iteration 3100 - loss 0.0041846097
train_accuracy: 99.9 ____ test_accuracy: 91.6  - epoch 40    iteration 3200 - loss 0.0039059598
train_accuracy: 99.95 ____ test_accuracy: 91.8  - epoch 42    iteration 3300 - loss 0.0031302233
train_accuracy: 99.95 ____ test_a

In [259]:
model.best_score

95.39999999980921

In [260]:
calculate_model_performance(np.argmax(Y_test, axis=0),
                           model.predict(X_test))

{'F1': 76.27118643890951,
 'accuracy': 94.3999999998112,
 'false_positive_rate': 5.947136563863553,
 'precision': 62.49999999913194,
 'prevalence': 9.1999999999816,
 'sensitivity/recall': 97.8260869543951,
 'specificity': 94.0528634359162}

In [261]:
calculate_model_performance(np.argmax(Y_validation, axis=0),
                           model.predict(X_validation))

{'F1': 78.0141843955807,
 'accuracy': 93.7999999998124,
 'false_positive_rate': 6.966292134815806,
 'precision': 63.95348837134938,
 'prevalence': 10.999999999978,
 'sensitivity/recall': 99.99999999818182,
 'specificity': 93.03370786495948}

In [262]:
model.best_W[1][0][7]

-0.0393697009874934

In [263]:
model.W[1][0][7]

-0.11189225771617269

In [266]:
model.W = model.best_W
model.B = model.best_B

In [267]:
calculate_model_performance(np.argmax(Y_test, axis=0),
                           model.predict(X_test))

{'F1': 79.64601769722266,
 'accuracy': 95.39999999980921,
 'false_positive_rate': 4.845814977962895,
 'precision': 67.16417910347516,
 'prevalence': 9.1999999999816,
 'sensitivity/recall': 97.8260869543951,
 'specificity': 95.15418502181684}

In [268]:
calculate_model_performance(np.argmax(Y_validation, axis=0),
                           model.predict(X_validation))

{'F1': 72.8476821177773,
 'accuracy': 91.79999999981641,
 'false_positive_rate': 9.21348314604671,
 'precision': 57.29166666606987,
 'prevalence': 10.999999999978,
 'sensitivity/recall': 99.99999999818182,
 'specificity': 90.78651685372857}

In [ ]:
# results_dict_all_models, results_average_dict, models = grid_search(
#     x,
#     y,
#     clf=NeuralNetwork,
#     lst_metrics=["F1", "accuracy"],
#     sort_by = "F1",
#     n_folds=10,
#     dict_param_grid={
#         'batch_size': [64, 128, 256],
#         'input_layer': [(x.shape[1], 'relu')],
#         'hidden_layer': [
#             [(50, 'relu'), (25, 'relu'), (10,'softmax')],
#             [(200, 'relu'), (100, 'relu'), (50, 'relu'), (10,'softmax')]
#         ],
#         'optimizer':[
#             {
#                 "method": "ADAM",
#                 "beta1": 0.9,
#                 "beta2": 0.999
#             }
#         ],
#         'output_layer': [10],
#         'alpha': [0.0001, 0.001],
#         'verbose': [False],
#         'epoch': [250]
#     }
# )

# results_average_dict
# print(models["model_1"])

In [ ]:
def display_miss_clasifications(model, digits_to_display):
    count = 0
    for index, (act, predicted) in enumerate(zip(np.argmax(Y,axis=0), model.predict(X))):
        if act != predicted:
            fig, ax = plt.subplots(figsize = (2,2))
            ax.set_title("%s: act %s --- predicted %s" %(index, act, predicted))
            ax.imshow(X[:, index].reshape(-1,20).T)
            ax.axis('off');
            count += 1
        if count == digits_to_display:
            break

In [95]:
display_miss_clasifications(model, 50)

NameError: name 'display_miss_clasifications' is not defined

In [148]:
weights = np.array([1,2,3])
# keep = weights[:]

In [149]:
np.delete(weights, 0)

array([2, 3])

In [150]:
keep

array([1, 2, 3])

In [113]:
keep = np.copy(weights)

In [114]:
np.delete(weights, 0)

array([2, 3])

In [115]:
keep

array([1, 2, 3])